In [2]:
import pandas as pd
import regex as re
import string
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import LinearRegression
from bs4 import BeautifulSoup
import requests


In [2]:
df_true=pd.read_csv('../datasets/True.csv')
df_true['class']=1
# df_true.head()
df_fake=pd.read_csv('../datasets/Fake.csv')
df_fake['class']=0
frames=[df_fake,df_true]
df=pd.concat(frames,axis=0)
df.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [3]:
df.columns
df=df.drop(["title","subject","date"],axis=1)

In [4]:
df.head

<bound method NDFrame.head of                                                     text  class
0      Donald Trump just couldn t wish all Americans ...      0
1      House Intelligence Committee Chairman Devin Nu...      0
2      On Friday, it was revealed that former Milwauk...      0
3      On Christmas day, Donald Trump announced that ...      0
4      Pope Francis used his annual Christmas Day mes...      0
...                                                  ...    ...
21412  BRUSSELS (Reuters) - NATO allies on Tuesday we...      1
21413  LONDON (Reuters) - LexisNexis, a provider of l...      1
21414  MINSK (Reuters) - In the shadow of disused Sov...      1
21415  MOSCOW (Reuters) - Vatican Secretary of State ...      1
21416  JAKARTA (Reuters) - Indonesia will buy 11 Sukh...      1

[44898 rows x 2 columns]>

In [5]:
df=df.sample(frac=1)
df.head

<bound method NDFrame.head of                                                     text  class
15861  MANILA (Reuters) - Two weeks ahead of a region...      1
3954   Fox Host Bret Baier found himself forced to re...      0
7089   (Reuters) - A federal judge on Tuesday blocked...      1
3384   WASHINGTON (Reuters) - The White House is hopi...      1
1926   HOUSTON (Reuters) - Thirteen Superfund sites, ...      1
...                                                  ...    ...
17148  So, the working people of America are basicall...      0
9234   WASHINGTON (Reuters) - A former senior officia...      1
12738  This week, liberals have been repeating their ...      0
9441   HANOI (Reuters) - U.S. President Barack Obama ...      1
19671  NAIROBI (Reuters) - Kenyan police fired tearga...      1

[44898 rows x 2 columns]>

In [6]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [7]:
def clean(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    # text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text


In [8]:
df["text"]=df["text"].apply(clean)

In [9]:
df["text"]

0        manila  reuters    two weeks ahead of a region...
1        fox host bret baier found himself forced to re...
2         reuters    a federal judge on tuesday blocked...
3        washington  reuters    the white house is hopi...
4        houston  reuters    thirteen superfund sites  ...
                               ...                        
44893    so  the working people of america are basicall...
44894    washington  reuters    a former senior officia...
44895    this week  liberals have been repeating their ...
44896    hanoi  reuters    u s  president barack obama ...
44897    nairobi  reuters    kenyan police fired tearga...
Name: text, Length: 44898, dtype: object

In [10]:
X=df["text"]
y=df["class"]
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.25)


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(X_train)
xv_test = vectorization.transform(X_test)

In [12]:
from sklearn.svm import LinearSVC
svclf=LinearSVC(dual=0,random_state=42)
svclf.fit(xv_train,y_train)

LinearSVC(dual=0, random_state=42)

In [13]:
svclf.score(xv_train,y_train)

0.9995545392450925

In [3]:
dfs=pd.read_json('../datasets/Sarcasm_Headlines_Dataset_v2.json',lines=True)
dfs.to_csv('sarcasm.csv',index=False)


In [4]:
dfs=pd.read_csv('./sarcasm.csv')
dfs=dfs[['headline','is_sarcastic']]

In [18]:
dfs.head(1)

,headline,is_sarcastic
0,thirtysomething scientists unveil doomsday clo...,1


In [17]:
dfs['headline']=dfs['headline'].apply(clean)

In [7]:
vectorize = TfidfVectorizer()
xs=dfs["headline"]
ys=dfs["is_sarcastic"]
Xs=vectorize.fit_transform(xs)
Xs_train,Xs_test,ys_train,ys_test=train_test_split(Xs,ys,random_state=42,test_size=0.25)

In [21]:
from sklearn.svm import LinearSVC
sarcsvc=LinearSVC()

sarcsvc.fit(Xs_train,ys_train)
def WebScrape(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser') 
    listOfParas=[]
    for i in soup.find_all('p'):
        listOfParas.append(i.text.strip())
    listOfSentences=[]
    for i in listOfParas:
        for sentence in i.split('.'):
            if sentence == "":
                continue
            listOfSentences.append(sentence)
    return listOfSentences

In [26]:
url=str(input())
def CalcSarcasmPercentage(url):
`    los=WebScrape(url)
    res=[]
    for x in los:       
        new=vectorize.transform([x]).toarray()
        ans=sarcsvc.predict(new)
        res.append(ans[0])
    count=0
    for y in res:
        if y==1:
            count+=1
    percsarc=(count/len(res))*100
    print(percsarc)`

60.0
